# ROI Analysis

The goal of an region-of-interest (ROI) analysis is to analyze the GLM parameter values for a region of interest. For each subject, we need to compute the mean paramter value (i.e. the regressor coefficient or beta value) of that ROI, which are then analyzed at the group level. Depending on our hypothesis, we can evaluate if there are differences between conditions, or simply if the ROI has beta values greater or less than 0. 

Why perform an ROI analysis if you have to run a GLM in the first place? Well, an ROI analysis assumes that you a pre-determined set of ROIs that you wish to examine and you can therefore avoid getting into whole-brain multiple-comparison stuff, which, as we learned, is pretty tricky. 

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nibabel as nib
from nilearn.maskers import NiftiMasker
from nilearn.plotting import plot_stat_map, plot_roi

import warnings
warnings.filterwarnings('ignore')

In [ ]:
with open('first_level_models.p', 'rb') as f:
    glms = pickle.load(f)

In [ ]:
glms

## 1. Combining parameter maps

First, let's select an example condition. For now, let's do Arms.

Then, we need to iterate through each subject and pull out their stat/parameter maps for Arms using the `compute_contrast` method we've been using.

In [ ]:
cond = 'Arm'

param_maps = []

for model in glms:
    parameter_map = model.compute_contrast(cond, output_type='effect_size')
    param_maps.append(parameter_map)

In [ ]:
param_maps

## 2. Viewing our ROIs

I've gone ahead and made some ROIs that will give us some meaningful resuts for the sake of this excercise. Let's plot each one:

In [ ]:
plot_roi('L_Broca.nii.gz')

In [ ]:
plot_roi('L_SSc.nii.gz')

In [ ]:
plot_roi('L_FEF.nii.gz')

## 3. Extracting data from an ROI

Okay, so now that we have our ROIs, we need to actually extract data from them. Nilearn has an amazing set of objects that make data extraction extremely easy. Masker objects are essential to the next several tutorials. I highly recommend looking at the [nilearn user guide](https://nilearn.github.io/stable/manipulating_images/masker_objects.html#masker-objects) to get a better understanding of what masker objects can do.

We will be using the core masker object, `NiftiMasker`. See the [documentation here](https://nilearn.github.io/stable/modules/generated/nilearn.maskers.MultiNiftiMasker.html#nilearn.maskers.MultiNiftiMasker). Notice that we can apply all sorts of preprocessing steps like we've done in with FirstLevelModels. We want the raw values from our images today, but in the following lessons we will be using the masker objects to also apply preprocessing steps. 

First, we initialize our masker object. We'll pass in one of our ROI masks. Then, we need to call the `fit_transform` method, which will fit the mask to the data and extract out the parameter values of the voxels that inside the ROI for each subject. This returns a matrix, where each row is a subject, and each column is a voxel. This has dimensions N x M, where N = the number of maps (subjects), and M = the number of voxels. You can think of masker objects 'flattening' the data into a 2D matrix, which is easy to work with.

In [ ]:
# set up masker
masker = NiftiMasker(mask_img='L_SSc.nii.gz')
# extract data
roi_data = masker.fit_transform(param_maps)

roi_data

In [ ]:
roi_data.shape

Next, we want to compute the mean parameter value for each subject. This will be our subject-level unit of data. We can conveniently do this using the numpy `mean` function, and set `axis=1`. This will compute the means across the columns (voxels) for each row.

In [ ]:
roi_means = np.mean(roi_data, axis=1)
roi_means

We should have 8 values; each value is the mean parameter value of Broca's area for each subject. 

In [ ]:
# Let's plot the roi_means across subjects

# Create the bar plot with x positions
x_positions = range(len(roi_means))
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.bar(x_positions, roi_means, color='skyblue')  # Use x positions and heights

# Add labels and title
plt.xlabel('Subject')
plt.ylabel('Mean Parameter Value (Beta)')
plt.title('Mean ROI Parameter Value per Subject') # Update title if condition is different
plt.ylim(min(0, min(roi_means)), max(0, max(roi_means)) * 1.2) # Adjust y-axis limits for better visualization, ensure 0 is included if relevant

# Add a horizontal line at 0 for reference (if betas can be negative/positive)
plt.axhline(0, color='black', linestyle='--', linewidth=0.8)

# Rotate x-axis labels if needed for better readability
plt.xticks(rotation=45, ha='right')
plt.tight_layout() # Adjust layout to prevent labels from overlapping

# Show the plot
plt.show()

## 4. Extracting all conditions

Now we'll repeat what we did above, but iterate through each condition. We'll save off our condition data in a dictionary (`condition_means`). Each item in the dictionary will the subjects' mean beta values for a condition. 

In [ ]:
conditions = ['Arm', 'Eye', 'Finger', 'Grasp', 'Mouth', 'Speech', 'Toes', 'Touch']
ROI = 'L_SSc.nii.gz'

condition_means = {} #let's loop over conditions
for cond in conditions:
    
    # get parameter map of each subject 
    param_maps = []
    for model in glms:
        parameter_map = model.compute_contrast(cond, output_type='effect_size')
        param_maps.append(parameter_map)
        
    # set mask and extract data
    masker = NiftiMasker(mask_img=ROI)
    roi_data = masker.fit_transform(param_maps)
    
    # compute mean for each subject and save off
    roi_means = np.mean(roi_data, axis=1)
    condition_means[cond] = roi_means
    

All our results are stored nicely in the dictionary:

In [ ]:
condition_means

## 5. Plotting the results

Finally, we want to plot our results so that we can interpret them. Instead of keeping all of our results in a dictionary, we can easily convert it to a pandas `DataFrame` object that we've seen before. Each key of the dictionary will become a column.

Putting things in a `DataFrame` not only lets us use Seaborn for plotting (see below), but it also lets us use the many powerful features provided by pandas. We'll use the `describe` method to look at the summary statistics before we plot.

In [ ]:
results = pd.DataFrame(condition_means)
results

In [ ]:
results.describe()

For plotting, we will want to reshape our data into a columnar, or 'long', format (many plotting libraries, like Seaborn, expect long-format data). 

Right now, each column is a condition, and each row is a subject. What we want is for each row to be a single beta value that is labelled according to it's condition. We don't need subject information for our analysis, so we don't need to worry about labeling each beta value according to the subject.

In [ ]:
# we will use the melt function in pandas to transform the dataframe from a wide format to a long (or tidy) format
results = results.melt(var_name='Condition', value_name='Beta')
results

We'll plot this using Seaborn, which is a fantastic statistical plotting library. The [online documentation is excellent](https://seaborn.pydata.org/index.html).

In [ ]:
sns.barplot(x='Condition', y='Beta', data=results, color='gray')

We can drastically improve this figure in order to make it publication-ready. We'll overlay individual data points, convert the 95% CI to standard error, fix up some styling stuff, and add a title. We'll bundle this all into a matplotlib figure (`fig`), which consists of a single panel (`ax`). We can edit properties of this panel by calling various methods.

How would you interpret this data? Change the ROI and repeat the above steps, and compare across regions.

In [ ]:
fig, ax = plt.subplots()
# over lay data points 
sns.stripplot(x='Condition', y='Beta', data=results, color='k', ax=ax, 
              edgecolor='k', marker='o', size=4)
# make nice bars
sns.barplot(x='Condition', y='Beta', data=results, color='dodgerblue', 
            ci=68, errcolor='k', errwidth=1.5, ax=ax)
# add a 0 line
ax.axhline(0, c='k', lw=1)
# add title and stylize the axes
ax.set_title('ROI analysis')
ax.xaxis.set_tick_params(length=0)
sns.despine(bottom=True)

## 6. Excercises: 

#### Exercise 1: Plot the results for the different ROIs
What conditions most strongly activate the L_Broca and L_FEF ROIs?

#### Exercise 2: Test whether the mean beta values differ significantly between two conditions for a specific ROI.

Select Two Conditions:
Choose two conditions (e.g., 'Arm' vs. 'Eye') for the ROI (e.g., L_SSc.nii.gz).

Extract Data:
Using the provided pipeline, extract the beta values for each subject under both conditions. Compute the subject-level mean beta values for each condition.

Perform a Statistical Test: Use a paired t-test to compare the mean beta values between the two conditions. 

Create a plot (such as a boxplot or a paired scatter plot) that displays the distribution of beta values for each condition.
Overlay individual subject data points.
Annotate the plot with the p-value from your statistical test.